In [32]:
from setup import init_django

In [33]:
init_django()

In [34]:
import helpers.clients as helper_clients

In [35]:
company_name: str = "Bitcoin"
ticker: str = "X:BTCUSD"
multiplier: int = 1
timespan:str = "minute"
from_date:str = "2025-07-30"
to_date:str = "2025-08-01"

client = helper_clients.PolygonAPIClient(
    ticker=ticker, 
    multiplier=multiplier,
    timespan=timespan,
    from_date=from_date, 
    to_date=to_date
)
dataset = client.get_stock_data()


In [36]:
dataset

[{'open_price': 117916.9,
  'close_price': 117920.26,
  'high_price': 118150,
  'low_price': 117916.9,
  'number_of_trades': 276,
  'volume': 1.297595839999999,
  'volume_weighted_average': 117934.353,
  'time': datetime.datetime(2025, 7, 30, 0, 0, tzinfo=<UTC>)},
 {'open_price': 117919.6,
  'close_price': 117892.76,
  'high_price': 118140,
  'low_price': 117882.49,
  'number_of_trades': 278,
  'volume': 2.21758317,
  'volume_weighted_average': 117906.7057,
  'time': datetime.datetime(2025, 7, 30, 0, 1, tzinfo=<UTC>)},
 {'open_price': 117892.76,
  'close_price': 117841.89,
  'high_price': 118110,
  'low_price': 117841.86,
  'number_of_trades': 266,
  'volume': 1.2186180999999996,
  'volume_weighted_average': 117875.8459,
  'time': datetime.datetime(2025, 7, 30, 0, 2, tzinfo=<UTC>)},
 {'open_price': 117841.88,
  'close_price': 117829.81,
  'high_price': 117900.1,
  'low_price': 117824.46,
  'number_of_trades': 237,
  'volume': 0.9826548100000003,
  'volume_weighted_average': 117844.4949

In [37]:
dataset[0]

{'open_price': 117916.9,
 'close_price': 117920.26,
 'high_price': 118150,
 'low_price': 117916.9,
 'number_of_trades': 276,
 'volume': 1.297595839999999,
 'volume_weighted_average': 117934.353,
 'time': datetime.datetime(2025, 7, 30, 0, 0, tzinfo=<UTC>)}

In [38]:
dataset[-1:]

[{'open_price': 113325.91,
  'close_price': 113248.73,
  'high_price': 113460,
  'low_price': 113235.04,
  'number_of_trades': 302,
  'volume': 1.0152746599999998,
  'volume_weighted_average': 113311.7132,
  'time': datetime.datetime(2025, 8, 1, 23, 59, tzinfo=<UTC>)}]

In [39]:
len(dataset)

4320

In [40]:
from market.models import Company, StockQuote

In [41]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=ticker)
company_obj

<Company: Company object (1)>

In [42]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (1)>, <StockQuote: StockQuote object (4)>, <StockQuote: StockQuote object (5)>, <StockQuote: StockQuote object (6)>, <StockQuote: StockQuote object (7)>, <StockQuote: StockQuote object (8)>, <StockQuote: StockQuote object (9)>, <StockQuote: StockQuote object (10)>, <StockQuote: StockQuote object (11)>, <StockQuote: StockQuote object (12)>, <StockQuote: StockQuote object (13)>, <StockQuote: StockQuote object (14)>, <StockQuote: StockQuote object (15)>, <StockQuote: StockQuote object (16)>, <StockQuote: StockQuote object (17)>, <StockQuote: StockQuote object (18)>, <StockQuote: StockQuote object (19)>, <StockQuote: StockQuote object (20)>, <StockQuote: StockQuote object (21)>, <StockQuote: StockQuote object (22)>, '...(remaining elements truncated)...']>

In [43]:
StockQuote.objects.all().count()

2880

In [44]:
# batching queries  
batch_size = 100
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(
            company=company_obj, **data
            )
        )
    print(i)
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300


In [45]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (1)>, <StockQuote: StockQuote object (4)>, <StockQuote: StockQuote object (5)>, <StockQuote: StockQuote object (6)>, <StockQuote: StockQuote object (7)>, <StockQuote: StockQuote object (8)>, <StockQuote: StockQuote object (9)>, <StockQuote: StockQuote object (10)>, <StockQuote: StockQuote object (11)>, <StockQuote: StockQuote object (12)>, <StockQuote: StockQuote object (13)>, <StockQuote: StockQuote object (14)>, <StockQuote: StockQuote object (15)>, <StockQuote: StockQuote object (16)>, <StockQuote: StockQuote object (17)>, <StockQuote: StockQuote object (18)>, <StockQuote: StockQuote object (19)>, <StockQuote: StockQuote object (20)>, <StockQuote: StockQuote object (21)>, <StockQuote: StockQuote object (22)>, '...(remaining elements truncated)...']>

In [46]:
StockQuote.objects.all().count()

5760